In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession, functions as f

# Reading Hospital A departments data 
df_hosa = spark.read.parquet("abfss://bronze@databricksdevetl.dfs.core.windows.net/hosa/departments")

# Reading Hospital B departments data 
df_hosb = spark.read.parquet("abfss://bronze@databricksdevetl.dfs.core.windows.net/hosb/departments")

# Union two departments DataFrames
df_merged = df_hosa.unionByName(df_hosb)

# Create Dept_id and rename deptid to SRC_Dept_id
df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid")) \
                     .withColumn("Dept_id", f.concat(f.col("deptid"), f.lit("-"), f.col("datasource"))) \
                     .drop("deptid")

# Register as a temp view
df_merged.createOrReplaceTempView("departments")

In [0]:
df_merged.display()

In [0]:
%sql
DROP TABLE IF EXISTS databricks_dev.silver.departments;

CREATE TABLE IF NOT EXISTS databricks_dev.silver.departments(
  Dept_Id string,
  SRC_Dept_Id string,
  Name string,
  datasource string,
  is_quarantined boolean
)
USING DELTA;

In [0]:
%sql
TRUNCATE TABLE databricks_dev.silver.departments

In [0]:
%sql
INSERT INTO TABLE databricks_dev.silver.departments
SELECT 
Dept_Id,
SRC_Dept_Id,
Name,
Datasource,
    CASE 
        WHEN SRC_Dept_Id IS NULL OR Name IS NULL THEN TRUE
        ELSE FALSE
    END AS is_quarantined
FROM departments;

In [0]:
%sql
SELECT * FROM databricks_dev.silver.departments;